In [ ]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
file = r"us-states.csv"
df = pd.read_csv(file, parse_dates=["date"])
df["cfr"] = df["deaths"]/df["cases"]

df = df.drop(columns=["fips"])

In [ ]:
last_updated = df["date"].max()
print("Last updated:", last_updated)
print("Time since first case:", df["date"].max() - df["date"].min())

In [ ]:
today = df.loc[df["date"] == last_updated, [
    "state", "cases", "deaths", "cfr"]]

def get_top(df, column, num_to_show=10):
    if column == "cfr":
        df = df.dropna()
    df = df[df["cfr"] <= 1]
    df = df.sort_values(by=column)
    lo = df.head(num_to_show)
    hi = df.tail(num_to_show).sort_values(by=column, ascending=False)
    return lo, hi

lo, hi = get_top(today, "deaths")
hi

In [ ]:
diff_1_suffix = " - Daily Increase"
diff_2_suffix = " - Change in Daily Increase"

window_size = 5
states = df.pivot(index="date", columns="state", values="cases")
state_names = states.columns

for state in state_names:
    states[state+diff_1_suffix] = states[state].diff().rolling(window_size, center=True).mean()
    states[state+diff_2_suffix] = states[state+diff_1_suffix].diff()
    
states

In [ ]:
import os

plt.rcParams['figure.figsize'] = [15, 10]

for state in state_names:
    df = states[[state, state+diff_1_suffix, state+diff_2_suffix]
             ].dropna()
    if not df.empty:
        df.plot.bar(subplots=True)
        os.makedirs(f'plots/{state}', exist_ok=True)
        plt.savefig(f'plots/{state}/overall.png', bbox_inches='tight')
        plt.close()

In [ ]:
daily_increase = states[[state+diff_1_postfix]]
weekday = daily_increase.groupby([daily_increase.index.weekday]).sum()
weekday.plot.bar()